# Case study 3: Flinders ranges
This example should be run using the Loop3d/Loop docker container. It requires 
* LoopStructural
* map2loop
* folium
* ipyleaflet
* ipywidgets
* lavavu-osmesa

It cannot be easily run on google colab, if you are using google colab run [this](South%20Australia%20Model%20No%20map2loop.ipynb) notebook instead.

In [ ]:
import folium
from ipyleaflet import Map, basemaps, GeoJSON, LayersControl, DrawControl,WMSLayer, GeoData
from ipywidgets import Label
from ipywidgets import Label
import ipywidgets as widgets
import numpy as np
from shapely.geometry import Polygon
import geopandas
import pandas as pd
import lavavu

In [ ]:
map_crs = 'EPSG:4326'
proj_crs = 'EPSG:28354'

Here we define a bounding box for the model, it will be displayed on a map in the next cell. You can use the map to draw your own bounding box if you want to try different model areas. 

**Warning** if you try new areas it may not work as expected because the modelling process assumes chronostratigraphic stratigraphy and requires adequate structural data. The structural data are shown by the red dots. 

In [ ]:
existing_extent = (138.362206, -32.463842, 139.262268, -31.999178)
center = (existing_extent[1]+existing_extent[3])/2, (existing_extent[0]+existing_extent[2])/2
minlat = existing_extent[3]
maxlat = existing_extent[1]
minlong = existing_extent[2]
maxlong = existing_extent[0]
lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
mbbox = geopandas.GeoDataFrame(index=[0], crs=map_crs, geometry=[bbox_geom])
example_rect = GeoData(geo_dataframe = mbbox,
                   style={'color': 'purple', 'opacity':3, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},                  
                   name = 'Example')
print(map_crs,mbbox.total_bounds)
mbbox=mbbox.to_crs(proj_crs)
print(proj_crs,mbbox.total_bounds)


In [ ]:
# center=(-33.5,138.7)

wms_warox = WMSLayer(
    url='http://geo.loop-gis.org/geoserver/GSSA/wms?',
    layers='GSSA:sth_flinders',
    format='image/png',
    transparent=True,
    attribution='Outcrop data from GSSA',
    name='outcrops'

)
wms_geol = WMSLayer(
    url='http://geo.loop-gis.org/geoserver/GSSA/wms?',
    layers='GSSA:2m surface geology',
    format='image/png',
    transparent=True,
    opacity=0.4,
    attribution='Geology data from GSSA',
    name='geology'

)
m =Map(basemap=basemaps.OpenTopoMap, center=center, zoom=8,scroll_wheel_zoom=True)
label = Label()
display(label)

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        label.value = str(kwargs.get('coordinates'))

m.on_interaction(handle_interaction)
m.add_layer(wms_geol)
m.add_layer(wms_warox)
# if(not test_data_name =='Draw Your Own'):
#     m.add_layer(example_rect)


m.add_control(LayersControl())
dc = DrawControl(rectangle={'shapeOptions': {'color': '#0000FF'}})
m.add_control(dc)
m.add_layer(example_rect)
m

In [ ]:
from geopandas import GeoDataFrame
from shapely.geometry import shape 
draw = dc.last_draw
try:
    poly = shape(draw['geometry'])
    poly_gdf = GeoDataFrame([1], geometry=[poly], crs=map_crs)
    bbox_gdf = poly_gdf.to_crs(proj_crs)
    bbox_3d = {"minx": np.min(bbox_gdf.loc[:,'geometry'][0].exterior.xy[0]),
         "miny": np.min(bbox_gdf.loc[:,'geometry'][0].exterior.xy[1]),
         "maxx": np.max(bbox_gdf.loc[:,'geometry'][0].exterior.xy[0]),
         "maxy": np.max(bbox_gdf.loc[:,'geometry'][0].exterior.xy[1]),
         "base": -3200,
         "top": 1200,}
except:
    minx = mbbox.total_bounds[0]
    maxx = mbbox.total_bounds[2]
    miny = mbbox.total_bounds[1]
    maxy = mbbox.total_bounds[3]# [ 274934.13200956 6297758.41393543  323980.90024616 6329307.64682912]
    bbox_3d = {'minx': minx,
     'miny': miny,
     'maxx': maxx,
     'maxy': maxy,
     'base': -3200,
     'top': 1200}

## Map2Loop
This process uses map2loop to process the goelogical datasets. Here we use local files.

In [ ]:
import os
import hjson
import map2loop
map2loop._clut_path='SA/data/GSSA_2M_colours.csv'
from map2loop.project import Project
proj = Project( 
                 structure_file='SA/data/sth_flinders_28354.shp',
                fault_file='SA/data/2M Linear Structures_28354.shp',
                fold_file='SA/data/2M Linear Structures_28354.shp',
                geology_file='SA/data/2M_Surface_Geology_28354_relage.shp',
                mindep_file="SA/data/null_mindeps.shp",
                 metadata='SA/data/meta.hjson',
#                  state = "SA",
#                  remote = False,
#                 clut_path='SA/data/GSSA_2M_colours.csv'
                 # path to hosted metadata describing the remote sources
#                  metadata='https://gist.githubusercontent.com/yohanderose/8f843de0dde531f009a3973cbdadcc9f/raw/918f412ae488ce1a6bca188306f7730061ecf551/meta_remote.hjson')                
)
proj.update_config(
                    out_dir='sa-test2',
                    bbox_3d=bbox_3d,
                    proj_crs={'init': 'EPSG:28354'},
#                     quiet=True,
#                     overwrite="true",
                    drift_prefix=['T','Q','water','void']
                  )

In [ ]:
proj.run()

## Loop Structural

In [ ]:
# Define project pathing from m2l
proj_path = proj.config.project_path
graph_path = proj.config.graph_path
tmp_path = proj.config.tmp_path
data_path = proj.config.data_path
dtm_path = proj.config.dtm_path
output_path = proj.config.output_path
vtk_path = proj.config.vtk_path

# Define project bounds
minx,miny,maxx,maxy = proj.config.bbox
model_base = proj.config.bbox_3d['base']
model_top = proj.config.bbox_3d['top']

fault_file = proj.config.fault_file_csv

In [ ]:
import random
import os
import time
from datetime import datetime
import shutil
import logging
logging.getLogger().setLevel(logging.INFO)

import numpy as np
from LoopStructural import GeologicalModel
import lavavu
from LoopStructural.visualisation import LavaVuModelViewer
from LoopStructural import GeologicalModel

f=open(os.path.join(tmp_path, 'bbox.csv'),'w')
f.write('minx,miny,maxx,maxy,lower,upper\n')
ostr='{},{},{},{},{},{}\n'.format(minx,miny,maxx,maxy,model_base,model_top)
f.write(ostr)
f.close()

fault_params = {'interpolatortype':'FDI',
                'nelements':3e5,
                'fault_buffer':.5,
                'solver':'pyamg',
#                 overprints:overprints,
                'cpw':10,
                'npw':10}
foliation_params = {'interpolatortype':'FDI' , # 'interpolatortype':'PLI', 'FDI', 'surfe'
                    'nelements':3e5,  # how many tetras/voxels
                    'buffer':1.8,  # how much to extend interpolation around box
                    'solver':'pyamg',
                    'damp':True,
                    'cpw':5,
                    'npw':5}

model, m2l_data = GeologicalModel.from_map2loop_directory(proj_path,
                                                          skip_faults=False,
                                                          rescale=False,
                                                          fault_params=fault_params,
                                                          foliation_params=foliation_params)




In [ ]:
view = LavaVuModelViewer(model,vertical_exaggeration=1) 
view.nsteps = np.array([200,200,100])
#view.set_zscale(2)
# view.add_model()
# view.nelements = 3e5#steps=np.array([,50,50])
#view.add_model_surfaces(filename=filename)
view.add_model_surfaces(opacity=0.8)
view.add_data(model['supergroup_0'])
view.interactive()  

In [ ]:
# to export the figure to webgl
# view.export_to_webgl('south_australia_model.html')

In [ ]:
geol_clip = proj.config.geol_clip
asc=pd.read_csv(tmp_path+'/all_sorts_clean.csv',",")
#display(asc)
colours=pd.read_csv(proj.config.clut_path,",")
if( proj.config.c_l['c']=='CODE'):
    code=proj.config.c_l['c'].lower()
else:
    code=proj.config.c_l['c']

colours = [] #container for the discrete colours we are using
i=0
geol_clip['colour_index'] = np.nan #initialise a colour index attribute column
for ind,strat in asc.iterrows():
    geol_clip[proj.config.c_l['c']].str.replace(" ","_")
    geol_clip.loc[geol_clip[proj.config.c_l['c']]==strat['code'].replace("_"," "),'colour_index'] = i
    colours.append(strat['colour'])
    i=i+1

In [ ]:
import matplotlib.pyplot as plt
from LoopStructural.visualisation import MapView
from LoopStructural.visualisation.stratigraphic_column import StratigraphicColumnView
from map2loop.map import MapUtil

In [ ]:
maps =proj.config.geol_clip#geopandas.read_file(proj.config.tmp_path+'geol_clip.shp')
masp = maps[maps['STRATNAM_1']!='Unnamed GIS Unit - see description']

extent=[model.bounding_box[0,0], model.bounding_box[1,0], model.bounding_box[0,1], model.bounding_box[1,1]]


mapview = MapView(model)
proj.config.load_dtm(True)
geomap = MapUtil(proj.config.bbox_3d,maps,proj.config.dtm)
mapview.nsteps = (200,200)
xy = np.array([mapview.xx
               .flatten(),mapview.yy.flatten()]).T
dtm = geomap.evaluate_dtm_at_points(xy)

geo_map = geomap.evaluate_geology_at_points(xy)
# dtm[:]=0
model_map = model.evaluate_model(np.vstack([xy.T,dtm]).T)

In [ ]:
data = []
colours = []
boundaries = []
for u, v in m2l_data['stratigraphic_column']['supergroup_0'].items():
    data.append((m2l_data['strat_va'][u],v['colour']))
    colours.append(v['colour'])
    boundaries.append(m2l_data['strat_va'][u])    
from matplotlib import colors
cmap = colors.ListedColormap(colours)
cmap = colors.ListedColormap(cmap.colors[::-1])
b = np.array(boundaries)
print()
b = b[::-1]
boundaries = colors.BoundaryNorm(b,ncolors=len(colours))

# Plot geology map and model comparison

In [ ]:
plt.rcParams.update({'font.size': 32})
fig3 = plt.figure(constrained_layout=True,figsize=(60,15))
nx = 6
gs = fig3.add_gridspec(5, 2*nx+1)
legend = fig3.add_subplot(gs[:-1, :1])
strike = fig3.add_subplot(gs[-1:,:1])

#add stike symbol
gradient_data = np.array([[0,1.,0]])
t = gradient_data[:, [1, 0]] * np.array([1, -1]).T
n = gradient_data[:, 0:2]
# t *= symb_scale
n *= 0.5
p1 = gradient_data[:, [0, 1]] - t
p2 = gradient_data[:, [0, 1]] + t
# plt.scatter(val[:,0],val[:,1],c='black')
strike.plot([p1[:, 0], p2[:, 0]], [p1[:, 1], p2[:, 1]], 'black')
p1 = gradient_data[:, [0, 1]]
p2 = gradient_data[:, [0, 1]] + n
strike.plot([p1[:, 0], p2[:, 0]], [p1[:, 1], p2[:, 1]], 'black')
strike.annotate("Bedding orientation", xy=[0,0],xytext=[1.5,.75],annotation_clip=False,size=32)
strike.set_xlim(-2,2)
# strike.plot()
strike.axis('off')
strike.axis('square')
p1 = gradient_data[:, [0, 1]] - t
p2 = gradient_data[:, [0, 1]] + t
strike.plot([p1[:, 0], p2[:, 0]], [p1[:, 1]+1.2, p2[:, 1]+1.2], 'black')
strike.annotate("Fault trace", xy=[0,1.2],xytext=[1.5,.75+1.2],annotation_clip=False,size=32)

# f3_ax1.set_title('gs[0, :-2]')
geol_map_ax = fig3.add_subplot(gs[:, 1:nx+1])
model_map_ax = fig3.add_subplot(gs[:, nx+1:])

column_ax = StratigraphicColumnView(model,legend)
geol_map_ax.imshow(geo_map.reshape(mapview.nsteps).T,extent=extent,cmap=cmap,origin='lower',vmin=0,vmax=7)
# proj.config.faults_clip.plot(ax=geol_map_ax)
mapview2 = MapView(model,ax=model_map_ax)
mapview2.nsteps=(200,200)
mapview2.add_data(model.features[-1],dip=False,val=False,symb_scale=600)
mapview2.add_model(cmap=cmap,z=dtm)
mapview2.add_faults(colors='black')
model_map_ax.imshow(model_map.reshape(mapview.nsteps).T,extent=extent,cmap=cmap,origin='lower',vmin=0,vmax=7)

legend.set_title('A. Legend',fontsize=40)
geol_map_ax.set_title('B. Geological map',fontsize=40)
model_map_ax.set_title('C. Interpolated geological map',fontsize=40)